In [1]:
import pandas as pd

In [2]:
eval=pd.read_csv('edd_eval_data.csv')
len(eval)

1352231

In [ ]:
 len(eval.dropna(axis=0, how='any',inplace=true))/len(eval)

In [4]:
eval.drop('fmc_days_to_delivery', axis=1, inplace=True)

In [370]:
len(eval.dropna())

1136460

In [372]:
eval.dropna(axis=0, how='any',inplace=True)

In [373]:
len(eval)

1136460

In [374]:
zip_mapping=pd.read_csv('DS0500.csv',low_memory=False)
zip_mapping.head()

zip_to_lat=dict(zip(zip_mapping.EVENZIP5,zip_mapping.Latitude))

zip_to_lon=dict(zip(zip_mapping.EVENZIP5,zip_mapping.Longitude))
 

In [375]:
zip_to_tz=dict(zip(zip_mapping.EVENZIP5,zip_mapping.TIMEZONE))

In [376]:
def des_lon(x):
    return zip_to_lon[x]

def des_lat(x):
    return zip_to_lat[x]

def des_tz(x):
    return zip_to_tz[x]

eval['des_lon']=eval['destination_zipcode'].apply(des_lon)
eval['des_lat']=eval['destination_zipcode'].apply(des_lat)
 
eval['des_tz']=eval['destination_zipcode'].apply(des_tz)

In [377]:
shipper=pd.read_csv('marego_shipper.csv')
shipper.head()

,shipper_id,origin_name,event_city,event_state,event_zipcode,event_lat,event_lng
0,102,Culture Kings,Chino,CA,91710,34.00507,-117.68249
1,102,mnml,Chino,CA,91710,34.00507,-117.68249
2,102,JLUXLABEL,Chino,CA,91710,34.00507,-117.68249
3,114,ALLBIRDS C/O UPS SCS,Shepherdsville,KY,40165,37.99407,-85.68918
4,117,ALLBIRDS C/O UPS SCS,Bloomington,CA,92316,34.05760,-117.39076


In [378]:
 eval.isnull().sum()

provider                  0
provider_status           0
provider_tracking_code    0
shipper_id                0
origin_name               0
                         ..
sin_weekday               0
origin_zip                0
des_tz                    0
distance_percentage       0
res_v_y                   0
Length: 65, dtype: int64

In [379]:
# map origin and destination zip to each row
# use merge maybe faster?

ori_lon=dict(zip(shipper.shipper_id,shipper.event_lng))

ori_lat=dict(zip(shipper.shipper_id,shipper.event_lat))

ori_zip=dict(zip(shipper.shipper_id,shipper.event_zipcode))

def ori_map_lon(x):
    try:
        return ori_lon[x]
    except: 
        return None
def ori_map_lat(x):
    try:
        return ori_lat[x]
    except:
        return None
   

def ori_map_zip(x):
    try:
        return ori_zip[x]
    except:
        return None
 
    

eval['origin_lon']=eval['shipper_id'].apply(ori_map_lon)
eval['origin_lat']=eval['shipper_id'].apply(ori_map_lat)
eval['origin_zip']=eval['shipper_id'].apply(ori_map_zip)


In [380]:
from math import sin
from math import cos
from math import atan2
from math import sqrt

def get_distance(lat1,lon1,lat2,lon2):
    R = 6370
    dlon = lon2 - lon1
    dlat = lat2- lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

eval['distance_ori']   = eval.apply(lambda x: get_distance(x.origin_lat, x.origin_lon,x.event_lat, x.event_lng) ,axis=1)
eval['distance_des']   = eval.apply(lambda x: get_distance(x.des_lat, x.des_lon,x.event_lat, x.event_lng) ,axis=1)


In [381]:
eval['distance_percentage']=eval['distance_des']/(eval['distance_des']+eval['distance_ori'])

In [382]:
from datetime import datetime
from dateutil import tz

In [383]:
from_zone = tz.gettz('UTC')

def get_local_tz(timezone,utc):
    to_zone = tz.gettz(timezone)
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

In [384]:
def get_timestamp(x):
    
    t=x.split('.')[0]
    #try:
    return datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    #except:
      #  return datetime(2022, 12, 31, 0, 0)
    
eval['format_event_date_time_local']=eval['event_date_time_local'].apply(get_timestamp)

In [385]:
eval['format_event_date_time_local']=eval['format_event_date_time_local'].apply(lambda x: x.replace(tzinfo=None))


In [386]:
import time
import datetime
import math
import numpy as np
 

eval['pracel_month']=eval['format_event_date_time_local'].apply(lambda x:x.month)

eval["month_norm"] = 2 * math.pi * eval['pracel_month'] / eval['pracel_month'].max()

eval["cos_month"] = np.cos(eval['month_norm'])
eval["sin_month"] = np.sin(eval['month_norm'])

In [387]:
eval['pracel_week']=eval['format_event_date_time_local'].apply((lambda x:x.isocalendar()[1]))

eval["week_norm"] = 2 * math.pi * eval['pracel_week'] / eval['pracel_week'].max()

eval["cos_week"] = np.cos(eval['week_norm'])
eval["sin_week"] = np.sin(eval['week_norm'])

In [388]:
eval['pracel_weekday']=eval['format_event_date_time_local'].apply((lambda x:x.weekday()))

eval["weekday_norm"] = 2 * math.pi * eval['pracel_weekday'] / eval['pracel_weekday'].max()

eval["cos_weekday"] = np.cos(eval['weekday_norm'])
eval["sin_weekday"] = np.sin(eval['weekday_norm'])

In [389]:
eval.columns

Index(['provider', 'provider_status', 'provider_tracking_code', 'shipper_id',
       'origin_name', 'maergo_internal_status', 'maergo_external_status',
       'maergo_tracking_code', 'maergo_message', 'event_date_time_utc',
       'event_date_time_local', 'event_time_zone', 'label_created_date_time',
       'event_city', 'event_state', 'event_zipcode', 'event_lat', 'event_lng',
       'destination_city', 'destination_state', 'destination_zipcode',
       'destination_country', 'hidden', 'distribution_center_id', 'client_id',
       'event_id', 'delivery_event_id', 'sort_center', 'sort_code',
       'origin_lon', 'origin_lat', 'distance_ori', 'des_lon', 'des_lat',
       'distance_des', 'format_event_date_time_local', 'pracel_month',
       'month_norm', 'cos_month', 'sin_month', 'pracel_week', 'week_norm',
       'cos_week', 'sin_week', 'y_eval_H', 'y_eval_V',
       'estimated_delivery_time_local', 'estimated_delivery_date_H',
       'estimated_delivery_date1_H', 'estimated_delivery_d

In [390]:
cat_fea=['provider_status' , 
         'origin_name' , 
         'maergo_external_status' , 
         'event_time_zone' ,
         'event_zipcode', 
         'destination_zipcode',
         'sort_code',
         
         'provider',
         'maergo_internal_status',
         'origin_zip',
         'shipper_id',
         'des_tz',
         
         
        ]

num_fea=[
 'event_lat',
 'event_lng',
 'des_lon',
 'des_lat',
 'origin_lon',
 'origin_lat',
     
 'distance_ori',
 'distance_des',
 'distance_percentage',
     
 'pracel_month',
 'cos_month',
 'sin_month',
     
 'pracel_weekday',                                        
 'cos_weekday',                        
 'sin_weekday',     
     
 'pracel_week',                                                 
 'cos_week',                           
 'sin_week',  
     
 ]
    
 

In [391]:
len(num_fea+cat_fea)

30

In [392]:
eval[num_fea+cat_fea]

eval['provider']=eval['provider'].astype("category")

 
eval['provider_status']=eval['provider_status'].astype("category")
eval['origin_name']=eval['origin_name'].astype("category")
eval['maergo_external_status']=eval['maergo_external_status'].astype("category")
 
eval['maergo_message']=eval['maergo_message'].astype("category")
 
eval['event_time_zone']=eval['event_time_zone'].astype("category")
eval['event_zipcode']=eval['event_zipcode'].astype("category")
eval['destination_city']=eval['destination_city'].astype("category")
eval['destination_state']=eval['destination_state'].astype("category")
eval['destination_zipcode']=eval['destination_zipcode'].astype("category")
eval['destination_country']=eval['destination_country'].astype("category")
eval['sort_code']=eval['sort_code'].astype("category")

eval['provider']=eval['provider'].astype("category")
eval['maergo_internal_status']=eval['maergo_internal_status'].astype("category")
eval['origin_zip']=eval['origin_zip'].astype("category")
eval['shipper_id']=eval['shipper_id'].astype("category")
eval['des_tz']=eval['des_tz'].astype("category")

In [393]:
eval.head()

,provider,provider_status,provider_tracking_code,shipper_id,origin_name,maergo_internal_status,maergo_external_status,maergo_tracking_code,maergo_message,event_date_time_utc,...,res_v,res_h,pracel_weekday,weekday_norm,cos_weekday,sin_weekday,origin_zip,des_tz,distance_percentage,res_v_y
0,OLX_PLD,Label Created,1LSCYJR001XVOOQ,102,mnml,Label Created,Label Created,XLX0360017C9K8F15,Label has been created,2023-06-14 17:17:46.344000,...,1,1,2,2.094395,-0.5,8.660254e-01,91710.0,America/New_York,1.00000,1.0
1,LASERSHIP,DLVD,1LSCYJR001V2YT2,102,That's So Fetch,Delivered,Delivered,XLX0360016X9F8HFS,Your package has been delivered,2023-06-01 21:27:24.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0
2,LASERSHIP,ARRD,1LSCYJR001V2YT2,102,That's So Fetch,In-Transit: Processing at Facility,In-Transit,XLX0360016X9F8HFS,Package is processing at facility,2023-06-01 12:29:15.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0
3,LASERSHIP,LOAD,1LSCYJR001V2YT2,102,That's So Fetch,In-Transit: Departed Facility,In-Transit,XLX0360016X9F8HFS,Package has departed facility,2023-06-01 14:40:31.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0
4,LASERSHIP,OFDL,1LSCYJR001V2YT2,102,That's So Fetch,Out for Delivery,Out for Delivery,XLX0360016X9F8HFS,Out for delivery,2023-06-01 14:40:40.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0


In [394]:
import lightgbm as lgbm

In [395]:
#horizontal model
#clf1.save_model('lgm_reg_0713_percentage.txt', num_iteration=clf1.best_iteration)
#load from model:

H_model = lgbm.Booster(model_file='lgm_reg_0719_h.txt')

In [396]:
y_eval_H = H_model.predict(eval[num_fea+cat_fea],alpha=0.8) #Create predictions on test set
 

In [397]:
eval['y_eval_H'] = y_eval_H

In [398]:

V_model = lgbm.Booster(model_file='lgm_reg_0719_V_retrain.txt')
 

In [399]:
y_eval_V=V_model.predict(eval[num_fea+cat_fea],alpha=0.8) #Create predictions on test set
 

In [400]:
eval['y_eval_V'] = y_eval_V

In [401]:
eval['y_eval_V']=eval['y_eval_V'].apply(np.round)
eval['y_eval_H']=eval['y_eval_H'].apply(np.round)

In [402]:
eval.head()

,provider,provider_status,provider_tracking_code,shipper_id,origin_name,maergo_internal_status,maergo_external_status,maergo_tracking_code,maergo_message,event_date_time_utc,...,res_v,res_h,pracel_weekday,weekday_norm,cos_weekday,sin_weekday,origin_zip,des_tz,distance_percentage,res_v_y
0,OLX_PLD,Label Created,1LSCYJR001XVOOQ,102,mnml,Label Created,Label Created,XLX0360017C9K8F15,Label has been created,2023-06-14 17:17:46.344000,...,1,1,2,2.094395,-0.5,8.660254e-01,91710.0,America/New_York,1.00000,1.0
1,LASERSHIP,DLVD,1LSCYJR001V2YT2,102,That's So Fetch,Delivered,Delivered,XLX0360016X9F8HFS,Your package has been delivered,2023-06-01 21:27:24.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0
2,LASERSHIP,ARRD,1LSCYJR001V2YT2,102,That's So Fetch,In-Transit: Processing at Facility,In-Transit,XLX0360016X9F8HFS,Package is processing at facility,2023-06-01 12:29:15.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0
3,LASERSHIP,LOAD,1LSCYJR001V2YT2,102,That's So Fetch,In-Transit: Departed Facility,In-Transit,XLX0360016X9F8HFS,Package has departed facility,2023-06-01 14:40:31.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0
4,LASERSHIP,OFDL,1LSCYJR001V2YT2,102,That's So Fetch,Out for Delivery,Out for Delivery,XLX0360016X9F8HFS,Out for delivery,2023-06-01 14:40:40.000000,...,2,2,3,3.141593,-1.0,1.224647e-16,91710.0,America/New_York,0.00256,2.0


In [403]:
# test model on training set to see if metrics is right

In [404]:
df=pd.read_csv('marego_preprocessed_0719.csv')
 

In [405]:
df=df[df.pracel_month!=12]
df=df[df.maergo_external_status!='Delivered']
df=df[df.maergo_external_status!='Out for Delivery']
df=df[df.maergo_external_status!='Exception']

In [406]:
#modify those columns, feature names, model


df=df[df['delivery_period']<8]
df=df[df['delivery_period']>0]

df['provider']=df['provider'].astype("category")

 
df['provider_status']=df['provider_status'].astype("category")
df['origin_name']=df['origin_name'].astype("category")
df['maergo_external_status']=df['maergo_external_status'].astype("category")
 
df['maergo_message']=df['maergo_message'].astype("category")
 
df['event_time_zone']=df['event_time_zone'].astype("category")
df['event_zipcode']=df['event_zipcode'].astype("category")
df['destination_city']=df['destination_city'].astype("category")
df['destination_state']=df['destination_state'].astype("category")
df['destination_zipcode']=df['destination_zipcode'].astype("category")
df['destination_country']=df['destination_country'].astype("category")
df['sort_code']=df['sort_code'].astype("category")

df['provider']=df['provider'].astype("category")
df['maergo_internal_status']=df['maergo_internal_status'].astype("category")
df['origin_zip']=df['origin_zip'].astype("category")
df['shipper_id']=df['shipper_id'].astype("category")
df['des_tz']=df['des_tz'].astype("category")

 
 
 
 
 




In [407]:
len(df)

2682475

In [408]:
df['distance_percentage']=df['distance_des']/(df['distance_des']+df['distance_ori'])

In [409]:
predV=V_model.predict(df[num_fea+cat_fea],alpha=0.8) 

In [410]:
df['yv']=predV
df['yv']=df['yv'].apply(np.round)
df['resv']=df['yv']-df['delivery_period']

In [411]:
def acc_res(res):
    lis=[]
    for i in res:
        if i<=1 and i>=0:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

In [412]:
acc_res(df['resv'])

0.8650559651068509

In [413]:
predH=H_model.predict(df[num_fea+cat_fea],alpha=0.8) 

In [414]:
df['yh']=predH
df['yh']=df['yh'].apply(np.round)
df['resh']=df['yh']-df['delivery_period']

In [415]:
acc_res(df['resh'])

0.8371336172750911

In [416]:
from datetime import timedelta
 

In [417]:
#eval['format_event_date_time_local'][1] + timedelta(days=1)

In [418]:
def get_delivery_day(event_day,period):    
    return event_day + timedelta(days=period)
 

In [419]:
eval['estimated_delivery_time_local_H'] = eval.apply(lambda x: get_delivery_day(x.format_event_date_time_local, x.y_eval_H ) ,axis=1)

In [420]:
eval.groupby('y_eval_H').apply(len)

y_eval_H
0.0        19
1.0     97434
2.0    232090
3.0    386533
4.0    272469
5.0    116547
6.0     27289
7.0      3852
8.0       217
9.0        10
dtype: int64

In [421]:
eval.groupby('y_eval_V').apply(len)

y_eval_V
1.0    178836
2.0    580328
3.0    256766
4.0    104969
5.0     15242
6.0       319
dtype: int64

In [422]:
df1=eval
df1.isnull().sum()

provider                           0
provider_status                    0
provider_tracking_code             0
shipper_id                         0
origin_name                        0
                                  ..
origin_zip                         0
des_tz                             0
distance_percentage                0
res_v_y                            0
estimated_delivery_time_local_H    0
Length: 66, dtype: int64

In [423]:
eval['estimated_delivery_time_local_V']   = eval.apply(lambda x: get_delivery_day(x.format_event_date_time_local, x.y_eval_V) ,axis=1)

In [424]:
eval[['format_event_date_time_local','y_eval_H','y_eval_V','estimated_delivery_date_H','estimated_delivery_time_local_V']].head(10)

,format_event_date_time_local,y_eval_H,y_eval_V,estimated_delivery_date_H,estimated_delivery_time_local_V
0,2023-06-14 10:17:46,4.0,3.0,2023-06-17 10:17:46,2023-06-17 10:17:46
1,2023-06-01 17:27:24,3.0,2.0,2023-06-03 17:27:24,2023-06-03 17:27:24
2,2023-06-01 08:29:15,2.0,2.0,2023-06-03 08:29:15,2023-06-03 08:29:15
3,2023-06-01 10:40:31,2.0,2.0,2023-06-03 10:40:31,2023-06-03 10:40:31
4,2023-06-01 10:40:40,3.0,2.0,2023-06-03 10:40:40,2023-06-03 10:40:40
5,2023-05-31 20:36:35,1.0,1.0,2023-06-01 20:36:35,2023-06-01 20:36:35
6,2023-06-12 19:19:35,4.0,3.0,2023-06-15 19:19:35,2023-06-15 19:19:35
7,2023-06-12 20:37:18,4.0,3.0,2023-06-15 20:37:18,2023-06-15 20:37:18
8,2023-05-31 20:32:15,4.0,3.0,2023-06-02 20:32:15,2023-06-03 20:32:15
9,2023-05-31 19:00:51,4.0,3.0,2023-06-02 19:00:51,2023-06-03 19:00:51


In [425]:
#eval_res=eval[["event_id",'estimated_delivery_time_local_H','estimated_delivery_time_local_V']]

In [426]:
eval = eval.rename(columns={ 'estimated_delivery_time_local_H':'estimated_delivery_date_H',  'estimated_delivery_time_local_V':'estimated_delivery_date_V'})

In [427]:
#eval_res = eval_res.rename(columns={ 'estimated_delivery_date1_H':'estimated_delivery_date_H','estimated_delivery_date_H':'estimated_delivery_date_V'})

In [428]:
eval_res=eval_res[["event_id",'estimated_delivery_date_H','estimated_delivery_date_V']]

In [429]:
eval = eval.loc[:,~eval.columns.duplicated()].copy()

In [430]:
eval.columns

Index(['provider', 'provider_status', 'provider_tracking_code', 'shipper_id',
       'origin_name', 'maergo_internal_status', 'maergo_external_status',
       'maergo_tracking_code', 'maergo_message', 'event_date_time_utc',
       'event_date_time_local', 'event_time_zone', 'label_created_date_time',
       'event_city', 'event_state', 'event_zipcode', 'event_lat', 'event_lng',
       'destination_city', 'destination_state', 'destination_zipcode',
       'destination_country', 'hidden', 'distribution_center_id', 'client_id',
       'event_id', 'delivery_event_id', 'sort_center', 'sort_code',
       'origin_lon', 'origin_lat', 'distance_ori', 'des_lon', 'des_lat',
       'distance_des', 'format_event_date_time_local', 'pracel_month',
       'month_norm', 'cos_month', 'sin_month', 'pracel_week', 'week_norm',
       'cos_week', 'sin_week', 'y_eval_H', 'y_eval_V',
       'estimated_delivery_time_local', 'estimated_delivery_date_H',
       'estimated_delivery_date1_H', 'estimated_delivery_d

In [431]:
eval_res.head(100)

,event_id,estimated_delivery_date_H,estimated_delivery_date_V
0,9a7eb8c8e80bcce1e6156f508deeb584,2023-06-17 10:17:46,2023-06-17 10:17:46
1,98e10437c2d5e7fd9486bf40e7247f8e,2023-06-03 17:27:24,2023-06-03 17:27:24
2,7fcabb105f6e458b75d98b1b735382f5,2023-06-03 08:29:15,2023-06-03 08:29:15
3,85e3343b00aeccef2490f7a72ed0f725,2023-06-03 10:40:31,2023-06-03 10:40:31
4,98450fc32b0c3ad4778dc7a5e60a7b42,2023-06-03 10:40:40,2023-06-03 10:40:40
...,...,...,...
95,dd987f5b63b7ad5abb43b16ade32ea79,2023-06-12 12:23:39,2023-06-11 12:23:39
96,e6fb495b0b00525074d3480fd087befe,2023-06-12 11:28:08,2023-06-11 11:28:08
97,82dc4b45a0f753d22f1a5d9387277495,2023-06-12 08:25:43,2023-06-11 08:25:43
98,f185f1d8c780b0fc15c375c71cb17ab0,2023-06-10 18:25:04,2023-06-09 18:25:04


In [432]:
eval.to_csv('marego_0717_eval_pred_V_H.csv')

In [433]:
eval_res.to_csv('marego_0717_eval_pred_result_V_H.csv')

In [434]:
eval_res[["event_id",'estimated_delivery_date_H']].to_csv('marego_0717_eval_pred_H.csv')

PermissionError: [Errno 13] Permission denied: 'marego_0717_eval_pred_H.csv'

In [ ]:
eval_res[["event_id",'estimated_delivery_date_V']].to_csv('marego_0717_eval_pred_V.csv')

In [ ]:
eval_res[["event_id",'estimated_delivery_date_V']]tc

In [ ]:
eval_true=pd.read_csv('eval_Labels.csv')
len(eval_true)

In [ ]:
dict_date=dict(zip(list(eval_true.event_id),list(eval_true.delivery_date_time_utc)))

In [ ]:
dict_y=dict(zip(list(eval_true.event_id),list(eval_true.days_to_delivery)))

In [ ]:
eval['date_true']=eval['event_id'].apply(lambda x:dict_date[x])

In [ ]:
eval['y_true']=eval['event_id'].apply(lambda x:dict_y[x])

In [ ]:
acc_res(eval['y_true'])

In [ ]:
eval.head(5)

In [ ]:
def get_timestamp(x):
    
    #t=x.split(' ')[0]
    try:
        return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    except:
        x=x.split('.')[0]
        return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [ ]:
eval['date_true_format']=eval['date_true'].apply(get_timestamp)

In [ ]:
zip_mapping=pd.read_csv('DS0500.csv',low_memory=False)
zip_mapping.head()

zip_to_timezone=dict(zip(zip_mapping.EVENZIP5,zip_mapping.TIMEZONE))

In [ ]:
def timezone_map(x):
    return zip_to_timezone[x]

In [ ]:
#eval['timezone']=eval['destination_zipcode'].apply(timezone_map)

In [ ]:
from_zone = tz.gettz('UTC')

def get_local_tz(timezone,utc):
    to_zone = tz.gettz(timezone)
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

In [ ]:
eval['format_date_true_local']   = eval.apply(lambda x: get_local_tz(x.des_tz, x.date_true_format) ,axis=1)

In [ ]:
#diff=eval['format_date_true_local'][1].date()-eval.estimated_delivery_time_local_V[1].date()

In [ ]:
diff.days

In [ ]:
eval.estimated_delivery_date1_H[1]

In [ ]:
eval.estimated_delivery_time_local_V[1]

In [ ]:
def date_diff(day1,day2):
    diff=day2.date()-day1.date()
    return diff.days

In [ ]:
eval['res_v']=eval.apply(lambda x:date_diff(x.format_date_true_local,x.estimated_delivery_date_V),axis=1)

In [ ]:
eval['res_h']=eval.apply(lambda x:date_diff(x.format_date_true_local,x.estimated_delivery_date_H),axis=1)

In [ ]:
eval['res_v_y']= eval['y_eval_V'] -eval['y_true']

In [ ]:
def acc_res(res):
    lis=[]
    for i in res:
        if i==0:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

def acc_res_1(res):
    lis=[]
    for i in res:
        if i>=0 and i<=1:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

def acc_res_2(res):
    lis=[]
    for i in res:
        if i>=0 and i<=2:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

def acc_res_3(res):
    lis=[]
    for i in res:
        if i>=0 and i<=3:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

def acc_res_4(res):
    lis=[]
    for i in res:
        if i>=0 and i<=4:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

def acc_res_5(res):
    lis=[]
    for i in res:
        if i>=0 and i<=5:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

def acc_res_(res):
    lis=[]
    for i in res:
        if i>=0:
            lis.append(1)
        else:
            lis.append(0)
    return sum(lis)/len(lis)

In [ ]:
eval1=eval[eval.maergo_external_status!='Exception']
eval1=eval1[eval1.pracel_month!=12]
eval1=eval1[eval1.maergo_external_status!='Delivered']
eval1=eval1[eval1.maergo_external_status!='Out for Delivery']
eval1=eval1[eval1.maergo_external_status!='Exception']

acc_res_1(eval1['res_v']) 

In [ ]:
acc_res_2(eval1['res_v']) 

 acc_res_2(eval1['res_v']) 

In [ ]:
#eval.groupby('maergo_external_status').agg({'res_v':acc_res_1})

In [ ]:
eval.groupby('maergo_external_status').agg({'res_v':[len,acc_res_1, acc_res_2, acc_res_3,acc_res_4,acc_res_5,acc_res_]}).reset_index()

In [ ]:
eval.groupby('maergo_external_status').agg({'res_h':[len,acc_res_1, acc_res_2, acc_res_3,acc_res_4,acc_res_5,acc_res_]}).reset_index()

In [ ]:
eval_test=eval[(eval.maergo_external_status=='In-Transit: Arrived at Sort Center') | (eval.maergo_external_status=='Label Created') ]

In [ ]:
eval_test.columns

In [ ]:
#look at performance on: 1) each stage, give acc_1, acc_2, .. acc_+

In [ ]:
eval_test=eval_test[['provider', 'provider_status', 'provider_tracking_code', 'shipper_id',
       'origin_name', 'maergo_internal_status', 'maergo_external_status',
       'maergo_tracking_code', 'maergo_message', 'event_date_time_utc',
       'event_date_time_local', 'event_time_zone', 'label_created_date_time',
       'event_city', 'event_state', 'event_zipcode', 'event_lat', 'event_lng',
       'destination_city', 'destination_state', 'destination_zipcode',
       'destination_country', 'hidden', 'distribution_center_id', 'client_id',
       'event_id', 'delivery_event_id', 'sort_center', 'sort_code',
       'origin_lon', 'origin_lat', 'distance_ori', 'des_lon', 'des_lat',
       'distance_des', 'format_event_date_time_local', 'pracel_month',
       'month_norm', 'cos_month', 'sin_month', 'pracel_week', 'week_norm',
       'cos_week', 'sin_week',    
       'estimated_delivery_time_local',  
        'estimated_delivery_date_V', 'y_true',
       'date_true', 'date_true_format',   'format_date_true_local',
       'res_v',   
        'y_eval_V']]

In [ ]:
eval_test.to_csv('evaluation_data.csv')

In [ ]:
 len(eval_test)

In [ ]:
 eval_test.groupby('maergo_external_status').apply(len)

In [ ]:
 eval.groupby('maergo_external_status').agg({'res_v': [len,acc_res_1,acc_res_2,acc_res_3,acc_res_]})

In [217]:
eval=eval[[ 'provider', 'provider_status', 'provider_tracking_code',
       'shipper_id', 'origin_name', 'maergo_internal_status',
       'maergo_external_status', 'maergo_tracking_code', 'maergo_message',
       'event_date_time_utc', 'event_date_time_local', 'event_time_zone',
       'label_created_date_time', 'event_city', 'event_state', 'event_zipcode',
       'event_lat', 'event_lng', 'destination_city', 'destination_state',
       'destination_zipcode', 'destination_country', 'hidden',
       'distribution_center_id', 'client_id', 'event_id', 'delivery_event_id',
       'sort_center', 'sort_code', 'origin_lon', 'origin_lat', 'distance_ori',
       'des_lon', 'des_lat', 'distance_des', 'format_event_date_time_local',
       'pracel_month', 'month_norm', 'cos_month', 'sin_month', 'pracel_week',
       'week_norm', 'cos_week', 'sin_week', 'y_eval_H', 'y_eval_V',
       'estimated_delivery_time_local', 'estimated_delivery_date_H',
       'estimated_delivery_date1_H', 'estimated_delivery_date_H',
       'estimated_delivery_time_local_V', 'y_true', 'date_true',
       'date_true_format', 'timezone', 'format_date_true_local', 'res_v',
       'res_h']]

In [ ]:
eval.to_csv('marego_evaluation_v_h_pred.csv')